### Training and Saving Intetn and NER models 

In [1]:
def train_intent(data,train_flag):
    #Importing the libraries for intent training
    import wget
    import numpy as np
    import json
    import numpy as np
    import os
    import pickle
    from sklearn.model_selection import train_test_split
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.preprocessing import OneHotEncoder,LabelEncoder
    #if loop to verify whether the ".txt" file is present or not, to satisfy the CI/CD pipeline
    if (not (os.path.exists('Data/glove.6B.100d.txt'))):
        print("inesle ")
        url= 'https://www.dropbox.com/s/a247ju2qsczh0be/glove.6B.100d.txt?dl=1'
        wget.download(url)
    #getting the json file keys from the data 
    keyss = list(data.keys())
    array11=np.array(data[keyss[0]])
    array12=np.array(data[keyss[1]])
    array13=np.array(data[keyss[2]])
    array14=np.array(data[keyss[3]])
    array15=np.array(data[keyss[4]])
    #Merging all the keys into one variable for training and testing
    val=np.concatenate([array11, array12,array13,array14,array15])
    text=[]
    labels=[]
    for i in  range(len(val)):
        text.append(val[i][0])
        labels.append(val[i][1])
    train_txt,test_txt,train_label,test_labels = train_test_split(text,labels,test_size = 0.2)
    #Implementing bag of words to collect all the words
    ls=[]
    for c in train_txt:
        ls.append(len(c.split()))
    maxLen=int(np.percentile(ls, 98))
    embeddings_index={}
    with open('Data/glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values=line.split()
            word=values[0]
            coefs=np.asarray(values[1:], dtype='float32')
            embeddings_index[word]=coefs
    all_embs=np.stack(embeddings_index.values())
    emb_mean, emb_std= all_embs.mean(), all_embs.std()
    max_num_words=40000
    embedding_dim=len(embeddings_index['the'])
    classes=np.unique(labels)
    tokenizer=Tokenizer(num_words=max_num_words)
    tokenizer.fit_on_texts(train_txt)
    train_sequences=tokenizer.texts_to_sequences(train_txt)
    train_sequences=pad_sequences(train_sequences, maxlen=maxLen, padding='post')
    test_sequences=tokenizer.texts_to_sequences(test_txt)
    test_sequences=pad_sequences(test_sequences, maxlen=maxLen, padding='post')
    word_index=tokenizer.word_index
    num_words=min(max_num_words, len(word_index))+1
    embedding_matrix=np.random.normal(emb_mean, emb_std, (num_words, embedding_dim))
    for word, i in word_index.items():
        if i >= max_num_words:
            break
        embedding_vector=embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    #Initialising One Hot Encoding
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(classes)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoder.fit(integer_encoded)
    train_label_encoded = label_encoder.transform(train_label)
    train_label_encoded = train_label_encoded.reshape(len(train_label_encoded), 1)
    train_label = onehot_encoder.transform(train_label_encoded)
    test_labels_encoded = label_encoder.transform(test_labels)
    test_labels_encoded = test_labels_encoded.reshape(len(test_labels_encoded), 1)
    test_labels = onehot_encoder.transform(test_labels_encoded)
    #Dumping the pickle file
    with open('Models/utils/tokenizer.pkl', 'wb') as file:
        pickle.dump(tokenizer, file)
    with open('Models/utils/label_encoder.pkl', 'wb') as file:
        pickle.dump(label_encoder, file)
    with open('Models/utils/classes.pkl', 'wb') as file:
        pickle.dump(classes, file)
    if train_flag == 1:
        intent_train_model(num_words,train_sequences,train_label,test_sequences,test_labels,embedding_matrix,classes)
    return("Intent training done")
    

In [2]:
def intent_train_model(num_words,train_sequences,train_label,test_sequences,test_labels,embedding_matrix,classes):
    #print("in training")
    #Importing libraries for feed forward neural network
    from keras.models import Sequential
    from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional,Embedding
    import pickle
    import json
    model = Sequential()
    model.add(Embedding(num_words, 100, trainable=False,input_length=train_sequences.shape[1], weights=[embedding_matrix]))
    model.add(Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.1, dropout=0.1), 'concat'))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(classes.shape[0], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    print(model.summary())
    history = model.fit(train_sequences, train_label, epochs = 15,
              batch_size = 20, shuffle=True,
              validation_data=[test_sequences, test_labels])
    model.save('models/intents.h5')
    return("Intent training done")
    
    

In [3]:
def train_ner(f): 
    import spacy
    from spacy.tokens import DocBin
    from tqdm import tqdm
    import json
    import os 
    if (os.path.exists('model-best')):
        nlp = spacy.load("model-best") # if already a trained model exists build upon it 
    else :
        nlp = spacy.blank("en") # if not load a blank spacy model
    db = DocBin()

    TRAIN_DATA = json.load(f)

    for text, annot in tqdm(TRAIN_DATA['annotations']): 
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents 
        db.add(doc)

    db.to_disk("training_data.spacy") # save the docbin object

    ! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

    ! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy
    
    return("Training done and New ner is ready")

In [4]:
#training all the models and save them 

import json
with open('Data/CW11.json') as file:
    data=json.loads(file.read())
train_flag=1
print(train_intent(data,train_flag))#training intent file 
f = open('Data/version4.json')
print(train_ner(f)) # traingin ner 

C:\Users\kesav\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3361: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 100)            12000     
                                                                 
 bidirectional (Bidirectiona  (None, 8, 512)           731136    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 8, 512)            0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 50)                12850     
                                                        

100%|████████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 2416.63it/s]



[x] The provided output file already exists. To force overwriting the config
file, set the --force or -F flag.



2022-08-15 14:12:56.854915: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-15 14:12:56.854995: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     62.94    0.00    0.00    0.00    0.00
 50     200         22.30   1324.64  100.00  100.00  100.00    1.00
111     400          0.00      0.00  100.00  100.00  100.00    1.00
178     600          0.00      0.00  100.00  100.00  100.00    1.00
274     800          0.00      0.00  100.00  100.00  100.00    1.00
374    1000          0.00      0.00  100.00  100.00  100.00    1.00
474    1200          0.00      0.00  100.00  100.00  100.00    1.00
663    1400          0.00      0.00  100.00  100.00  100.00    1.00
863    1600          0.00 

2022-08-15 14:13:10.385476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-08-15 14:13:10.385555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-08-15 14:13:21,776] [INFO] Set up nlp object from config
[2022-08-15 14:13:21,804] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-08-15 14:13:21,814] [INFO] Created vocabulary
[2022-08-15 14:13:21,817] [INFO] Finished initializing nlp object
[2022-08-15 14:13:22,056] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
